In [0]:

-- MV 1: Top Brands Performance
CREATE OR REPLACE MATERIALIZED VIEW `na-dbxtraining`.biju_gold.top_brands_mv
AS
SELECT 
  brand,
  SUM(total_revenue) as total_revenue,
  SUM(order_count) as total_orders,
  SUM(unique_customers) as total_customers,
  ROUND(SUM(total_revenue) / SUM(order_count), 2) as avg_order_value
FROM `na-dbxtraining`.biju_gold.sales_by_brand_category
GROUP BY brand
ORDER BY total_revenue DESC;


-- MV 2: Category Performance Summary
CREATE OR REPLACE MATERIALIZED  VIEW `na-dbxtraining`.biju_gold.category_summary_mv
AS
SELECT 
  category,
  SUM(total_revenue) as total_revenue,
  SUM(order_count) as total_orders,
  SUM(total_quantity) as total_items_sold,
  SUM(unique_customers) as total_customers,
  ROUND(AVG(avg_order_value), 2) as avg_order_value
FROM `na-dbxtraining`.biju_gold.sales_by_brand_category
GROUP BY category
ORDER BY total_revenue DESC;

-- MV 3: Location Sales Ranking
CREATE OR REPLACE MATERIALIZED VIEW `na-dbxtraining`.biju_gold.location_ranking_mv
AS
SELECT 
  location,
  total_revenue,
  order_count,
  unique_customers,
  RANK() OVER (ORDER BY total_revenue DESC) as revenue_rank,
  ROUND(total_revenue / order_count, 2) as avg_order_value,
  delivery_rate_pct
FROM `na-dbxtraining`.biju_gold.location_performance
ORDER BY revenue_rank;

-- MV 5: Customer Segmentation Summary
CREATE OR REPLACE  MATERIALIZED VIEW `na-dbxtraining`.biju_gold.customer_segments_mv
AS
SELECT 
  customer_segment,
  COUNT(*) as customer_count,
  ROUND(AVG(total_spent), 2) as avg_total_spent,
  ROUND(AVG(order_count), 2) as avg_order_count,
  ROUND(AVG(avg_order_value), 2) as avg_order_value,
  SUM(total_spent) as segment_total_revenue,
  ROUND(SUM(total_spent) * 100.0 / SUM(SUM(total_spent)) OVER (), 2) as revenue_contribution_pct
FROM `na-dbxtraining`.biju_gold.customer_insights
GROUP BY customer_segment
ORDER BY segment_total_revenue DESC;


-- MV 8: Discount Analysis
CREATE OR REPLACE  MATERIALIZED VIEW `na-dbxtraining`.biju_gold.discount_analysis_mv
AS
SELECT 
  product_id,
  product_name,
  brand,
  category,
  avg_discount_pct,
  total_discount_amount,
  total_revenue,
  ROUND(total_discount_amount / (total_revenue + total_discount_amount) * 100, 2) as discount_rate_pct,
  order_count
FROM `na-dbxtraining`.biju_gold.product_performance
WHERE avg_discount_pct > 0
ORDER BY total_discount_amount DESC;


--  MV 6: High Value Customers
CREATE OR REPLACE  MATERIALIZED VIEW `na-dbxtraining`.biju_gold.vip_customers_mv
AS
SELECT 
  customer_id,
  customer_name,
  location,
  total_spent,
  order_count,
  avg_order_value,
  customer_segment,
  SIZE(brands_purchased) as brands_count,
  SIZE(categories_purchased) as categories_count
FROM `na-dbxtraining`.biju_gold.customer_insights
WHERE customer_segment = 'High Value'
ORDER BY total_spent DESC;

-- MV 7: Brand-Location Performance Matrix
CREATE OR REPLACE  MATERIALIZED VIEW `na-dbxtraining`.biju_gold.brand_location_matrix_mv
AS
SELECT 
  s.brand,
  s.category,
  s.total_revenue as brand_revenue,
  l.location,
  l.total_revenue as location_revenue,
  ROUND(s.total_revenue / l.total_revenue * 100, 2) as brand_pct_of_location
FROM `na-dbxtraining`.biju_gold.sales_by_brand_category s
CROSS JOIN `na-dbxtraining`.biju_gold.location_performance l
WHERE s.total_revenue > 1000
ORDER BY s.brand, l.total_revenue DESC;

-- MV 8: Discount Analysis
CREATE OR REPLACE  MATERIALIZED VIEW `na-dbxtraining`.biju_gold.discount_analysis_mv
AS
SELECT 
  product_id,
  product_name,
  brand,
  category,
  avg_discount_pct,
  total_discount_amount,
  total_revenue,
  ROUND(total_discount_amount / (total_revenue + total_discount_amount) * 100, 2) as discount_rate_pct,
  order_count
FROM `na-dbxtraining`.biju_gold.product_performance
WHERE avg_discount_pct > 0
ORDER BY total_discount_amount DESC;
